In [ ]:
import pandas as pd
import numpy as np
import os

CONFIG = {
    "arquivo_entrada": "../data/raw/enade/microdados2023_arq1.txt", 
    "sep": ";",            
    "encoding": "latin-1", 
    
    # Seleção de Colunas: Carregamos SÓ o necessário para economizar RAM.
    # Adicione ou remova colunas aqui conforme sua análise evoluir.
    "colunas_interesse": [
        "NU_ANO",           # Ano
        "CO_CURSO",         # Código do Curso
        "NO_CURSO",         # Nome do Curso (se houver) ou TP_GRUPO
        "CO_MUNICIPIO_CURSO", # Para cruzar com IDH
        "CO_UF_CURSO",      # Estado da faculdade
        "TP_SEXO",          # Demografia
        "NU_IDADE",         # Demografia
        "NT_GER",           # Nota Geral (Nossa variável ALVO)
        "TP_PRES",          # Presença (para filtrar faltantes)
        "QE_I08",           # Renda (Geralmente a pergunta 8)
        "QE_I16",           # UF Ensino Médio (Para análise de migração)
        "CO_CATEGAD"        # Categoria (Pública/Privada)
    ]
}

def carregar_dados_enade(config):
    """
    Carrega os dados do Enade de forma otimizada.
    """
    print(f"🔄 Iniciando leitura do arquivo: {config['arquivo_entrada']}...")
    
    try:
       
        if not os.path.exists(config['arquivo_entrada']):
            raise FileNotFoundError(f"Arquivo não encontrado: {config['arquivo_entrada']}")

        df = pd.read_csv(
            config['arquivo_entrada'],
            sep=config['sep'],
            encoding=config['encoding'],
            usecols=lambda c: c in config['colunas_interesse'], 
            decimal=',' 
        )
        
        print(f"✅ Sucesso! Dados carregados.")
        print(f"📊 Dimensões: {df.shape[0]} linhas x {df.shape[1]} colunas")
        return df

    except Exception as e:
        print(f"❌ Erro fatal na leitura: {e}")
        return None

df = carregar_dados_enade(CONFIG)

if df is not None:
    display(df.head())

In [ ]:
# --- CONFIGURAÇÃO DE LIMPEZA ---
FILTROS = {
    "codigo_presenca_presente": 555,   # 555 = Presente (Padrão INEP)
    "coluna_presenca": "TP_PRES",      # Ajuste se no seu CSV for TP_PR_GER
    "coluna_nota": "NT_GER"
}

def higienizar_dados(df, filtros):
    """
    Aplica regras de negócio para garantir que só analisaremos 
    alunos que realmente fizeram a prova.
    """
    print("🧹 Iniciando higienização dos dados...")
    
    # 1. Cópia de segurança para não afetar o dataframe original na memória se der erro
    df_clean = df.copy()
    initial_rows = len(df_clean)
    
    # 2. Filtrar apenas presentes (Evita viés de absenteísmo)
    # Verifica se a coluna de presença existe antes de filtrar
    col_pres = filtros['coluna_presenca']
    if col_pres in df_clean.columns:
        df_clean = df_clean[df_clean[col_pres] == filtros['codigo_presenca_presente']]
        print(f"   - Removidos {initial_rows - len(df_clean)} alunos ausentes/desclassificados.")
    else:
        print(f"   ⚠️ Aviso: Coluna '{col_pres}' não encontrada. Pulei filtro de presença.")

    # 3. Remover notas nulas
    # Às vezes o aluno foi, mas a nota não foi computada (anulada, etc)
    rows_before_drop = len(df_clean)
    df_clean = df_clean.dropna(subset=[filtros['coluna_nota']])
    print(f"   - Removidos {rows_before_drop - len(df_clean)} alunos com nota nula (NaN).")
    
    # 4. Garantir tipo numérico (Defesa extra)
    df_clean[filtros['coluna_nota']] = pd.to_numeric(df_clean[filtros['coluna_nota']], errors='coerce')
    
    print(f"✅ Dados Limpos! Restaram {len(df_clean)} registros ({len(df_clean)/initial_rows:.1%} do original).")
    
    return df_clean

# --- EXECUÇÃO ---
df_final = higienizar_dados(df, FILTROS)

# Sanity Check (Verificação de Sanidade)
# Vamos ver se a estatística descritiva faz sentido (Mínimo 0, Máximo 100)
print("\n📊 Estatísticas da Nota Geral:")
display(df_final['NT_GER'].describe().round(2))